# TSP Slicer Demo

This notebook demonstrates the TSP (Traveling Salesman Problem) slicer module from Adaptive Dynamics Toolkit for 3D printing toolpath optimization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
from time import time

# Import the TSP slicer module
from adaptive_dynamics.tsp.slicer import Slicer, SliceConfig, OptimizationMethod

## 1. Configure the Slicer

First, let's configure our slicer with appropriate parameters for 3D printing.

In [ ]:
# Create slicer configuration
config = SliceConfig(
    layer_height=0.2,  # mm
    wall_thickness=1.2,  # mm
    infill_density=0.15,  # 15% infill
    optimization_method=OptimizationMethod.TWO_OPT,  # Use 2-opt algorithm for path optimization
    max_optimization_time=5.0,  # seconds
    adaptive_layer_height=True,  # Use adaptive layer heights
    min_layer_height=0.1,  # mm
    max_layer_height=0.3  # mm
)

# Create slicer instance
slicer = Slicer(config)

# Print configuration
print("Slicer Configuration:")
print(f"  Layer Height: {config.layer_height} mm")
print(f"  Wall Thickness: {config.wall_thickness} mm")
print(f"  Infill Density: {config.infill_density * 100}%")
print(f"  Optimization Method: {config.optimization_method.value}")
print(f"  Adaptive Layer Height: {config.adaptive_layer_height}")

## 2. Load a 3D Model

For this demo, we'll use a mock 3D model since we don't have access to actual STL files in this environment.

In [ ]:
# Create a temporary file path for demo purposes
mock_stl_path = "mock_model.stl"

# Load the model (this is a mock implementation)
start_time = time()
slicer.load_mesh(mock_stl_path)
load_time = time() - start_time

print(f"Loaded model in {load_time:.3f} seconds")
print(f"Model bounds: {slicer.mesh['bounds']}")

## 3. Generate Slices

Now, let's slice the model into horizontal layers.

In [ ]:
# Generate slices
start_time = time()
num_layers = slicer.generate_slices()
slice_time = time() - start_time

print(f"Generated {num_layers} layers in {slice_time:.3f} seconds")
print(f"Z range: {slicer.layers[0]['z']:.2f} mm to {slicer.layers[-1]['z']:.2f} mm")

## 4. Visualize Slices

Let's visualize a few of the generated slices to see the contours.

In [ ]:
def plot_layer(layer, title=None):
    """Plot a single layer's contours."""
    plt.figure(figsize=(8, 8))
    
    # Plot each contour in the layer
    for i, contour in enumerate(layer["contours"]):
        plt.plot(contour[:, 0], contour[:, 1], 'b-', linewidth=2, 
                 label=f"Contour {i+1}" if i < 3 else None)
    
    plt.axis('equal')
    plt.grid(True)
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    
    if title:
        plt.title(title)
    else:
        plt.title(f"Layer at Z = {layer['z']:.2f} mm")
    
    # Only show legend if we have a reasonable number of contours
    if len(layer["contours"]) <= 3:
        plt.legend()
    
    plt.show()

# Visualize a few layers
layers_to_show = [0, len(slicer.layers)//2, len(slicer.layers)-1]  # First, middle, last

for layer_idx in layers_to_show:
    layer = slicer.layers[layer_idx]
    plot_layer(layer, f"Layer {layer_idx+1} of {len(slicer.layers)} (Z = {layer['z']:.2f} mm)")

## 5. Optimize Toolpaths

Now, let's optimize the toolpaths for each layer to minimize travel movements.

In [ ]:
# Optimize paths
start_time = time()
total_segments = slicer.optimize_paths()
optimization_time = time() - start_time

print(f"Optimized paths with {total_segments} segments in {optimization_time:.3f} seconds")

## 6. Visualize Optimized Toolpaths

Let's visualize the optimized toolpaths for a few layers.

In [ ]:
def plot_optimized_layer(layer, title=None):
    """Plot a layer's contours and optimized path."""
    plt.figure(figsize=(10, 8))
    
    # Plot contours
    for contour in layer["contours"]:
        plt.plot(contour[:, 0], contour[:, 1], 'b-', alpha=0.3, linewidth=1)
    
    # Plot optimized path
    path = layer["paths"]
    if len(path) > 0:
        plt.plot(path[:, 0], path[:, 1], 'r-', linewidth=2, label="Optimized Path")
        plt.plot(path[0, 0], path[0, 1], 'go', markersize=8, label="Start Point")
        plt.plot(path[-1, 0], path[-1, 1], 'ro', markersize=8, label="End Point")
    
    plt.axis('equal')
    plt.grid(True)
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    
    if title:
        plt.title(title)
    else:
        plt.title(f"Optimized Path for Layer at Z = {layer['z']:.2f} mm")
    
    plt.legend()
    plt.show()

# Visualize optimized paths for a few layers
for layer_idx in layers_to_show:
    layer = slicer.layers[layer_idx]
    plot_optimized_layer(layer, f"Optimized Layer {layer_idx+1} (Z = {layer['z']:.2f} mm)")

## 7. Visualize Multiple Layers in 3D

Let's create a 3D visualization showing multiple layers stacked.

In [ ]:
def plot_3d_layers(layers, num_layers=5):
    """Plot multiple layers in 3D."""
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Select layers to show
    step = max(1, len(layers) // num_layers)
    layers_to_plot = layers[::step]
    if len(layers_to_plot) > num_layers:
        layers_to_plot = layers_to_plot[:num_layers]
    
    # Generate a color map for layers
    colors = plt.cm.viridis(np.linspace(0, 1, len(layers_to_plot)))
    
    for i, layer in enumerate(layers_to_plot):
        z = layer["z"]
        
        # Plot contours
        for contour in layer["contours"]:
            x = contour[:, 0]
            y = contour[:, 1]
            z_array = np.full_like(x, z)
            ax.plot(x, y, z_array, color=colors[i], alpha=0.7, linewidth=1)
        
        # Plot optimized path if available
        path = layer.get("paths", [])
        if len(path) > 0:
            x = path[:, 0]
            y = path[:, 1]
            z_array = np.full_like(x, z)
            ax.plot(x, y, z_array, 'r-', linewidth=2, alpha=0.8)
    
    ax.set_xlabel('X (mm)')
    ax.set_ylabel('Y (mm)')
    ax.set_zlabel('Z (mm)')
    ax.set_title('3D View of Selected Layers')
    
    # Create a legend for layer colors
    from matplotlib.lines import Line2D
    legend_elements = []
    for i, layer in enumerate(layers_to_plot):
        legend_elements.append(Line2D([0], [0], color=colors[i], lw=2, 
                                     label=f"Z = {layer['z']:.1f} mm"))
    ax.legend(handles=legend_elements, loc='upper right')
    
    plt.tight_layout()
    plt.show()

# Plot 3D view of multiple layers
plot_3d_layers(slicer.layers, num_layers=5)

## 8. Export G-code

Now, let's export the optimized toolpaths as G-code for 3D printing.

In [ ]:
# Define output file path
gcode_path = "output_model.gcode"

# Export G-code
start_time = time()
success = slicer.export_gcode(gcode_path)
export_time = time() - start_time

if success:
    print(f"G-code successfully exported to {gcode_path} in {export_time:.3f} seconds")
else:
    print("Failed to export G-code")

## 9. Print Statistics

Finally, let's print some statistics about the sliced model.

In [ ]:
# Get statistics
stats = slicer.get_statistics()

# Print formatted statistics
print("\nModel Statistics:")
print(f"  Number of Layers: {stats['num_layers']}")
print(f"  Total Path Length: {stats['total_path_length_mm']:.2f} mm")
print(f"  Total Path Segments: {stats['total_path_segments']}")

# Calculate print time in a readable format
print_time_s = stats["estimated_print_time_s"]
hours = int(print_time_s // 3600)
minutes = int((print_time_s % 3600) // 60)
seconds = int(print_time_s % 60)
print(f"  Estimated Print Time: {hours}h {minutes}m {seconds}s")

# Calculate filament length
filament_diameter_mm = 1.75
filament_volume_mm3 = stats["estimated_filament_volume_mm3"]
filament_length_mm = filament_volume_mm3 / (np.pi * (filament_diameter_mm/2)**2)
print(f"  Estimated Filament Length: {filament_length_mm/1000:.2f} m")
print(f"  Estimated Filament Volume: {filament_volume_mm3/1000:.2f} cm³")

# Assuming PLA density of 1.24 g/cm³
pla_density_g_cm3 = 1.24
filament_weight_g = (filament_volume_mm3 / 1000) * pla_density_g_cm3
print(f"  Estimated Filament Weight: {filament_weight_g:.2f} g (assuming PLA)")

## 10. Compare Optimization Methods

Let's compare different path optimization methods to see how they perform.

In [ ]:
def compare_optimization_methods(layer_idx=10):
    """Compare different optimization methods on a single layer."""
    methods = [
        OptimizationMethod.NEAREST_NEIGHBOR,
        OptimizationMethod.TWO_OPT
    ]
    
    # Get the selected layer
    layer = slicer.layers[layer_idx]
    contours = layer["contours"]
    
    # Extract all points that need to be visited
    all_points = []
    for contour in contours:
        step = max(1, len(contour) // 10)  # Take ~10 points per contour
        sampled_points = contour[::step]
        all_points.extend(sampled_points)
    
    all_points = np.array(all_points)
    
    # Plot comparison
    fig, axes = plt.subplots(1, len(methods), figsize=(15, 5))
    
    for i, method in enumerate(methods):
        ax = axes[i]
        
        # Create a temporary slicer with this method
        temp_config = SliceConfig(optimization_method=method)
        temp_slicer = Slicer(temp_config)
        
        # Optimize the path using this method
        start_time = time()
        if method == OptimizationMethod.NEAREST_NEIGHBOR:
            path = temp_slicer._nearest_neighbor_tsp(all_points)
        elif method == OptimizationMethod.TWO_OPT:
            path = temp_slicer._two_opt_tsp(all_points)
        else:
            path = temp_slicer._nearest_neighbor_tsp(all_points)
        
        optimization_time = time() - start_time
        
        # Calculate path length
        path_length = 0.0
        for j in range(1, len(path)):
            path_length += np.sqrt(np.sum((path[j] - path[j-1])**2))
        
        # Plot contours
        for contour in contours:
            ax.plot(contour[:, 0], contour[:, 1], 'b-', alpha=0.2)
        
        # Plot path
        ax.plot(path[:, 0], path[:, 1], 'r-', linewidth=1.5)
        ax.plot(path[0, 0], path[0, 1], 'go', markersize=6)
        
        ax.set_title(f"{method.value}\nTime: {optimization_time:.3f}s, Length: {path_length:.1f}mm")
        ax.set_aspect('equal')
        ax.grid(True)
    
    plt.tight_layout()
    plt.show()

# Compare optimization methods
compare_optimization_methods()

## Conclusion

In this notebook, we've demonstrated the capabilities of the Adaptive Dynamics Toolkit's TSP Slicer module for 3D printing toolpath optimization. Key features include:

1. Slicing 3D models into horizontal layers
2. Optimizing toolpaths using various TSP algorithms
3. Exporting G-code for 3D printing
4. Providing print statistics and visualizations

For real-world applications, you'd want to use a proper 3D mesh library to handle STL/OBJ files and implement more sophisticated slicing algorithms, but this demonstrates the core concepts behind the module.